In [98]:
from pyod.models.knn import KNN
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [99]:
#44009 and 44065 were the two buoys lying right on the trajectory of the hurricane 
#so we decided to use them to analyse the data further
df_b7_init = pd.read_csv('/Users/shreyabanerjee/DA_proj/clean_csvs/44009_clean.csv')
df_b8_init = pd.read_csv('/Users/shreyabanerjee/DA_proj/clean_csvs/44065_clean.csv')

#adding the two dataframes
df=pd.concat([df_b7_init,df_b8_init],axis=0)

In [100]:
#the readings that actually contribute in prediction of hurricane
keep_col=['pressure','windspeed']
X = df[keep_col].values
y = df['possibility'].values


In [101]:
X.shape

(16651, 2)

In [102]:
y.shape

(16651,)

In [103]:
#While making the csvs for each buoy individually, we observed that occurence of hurricane is related to pressure drop
#So we made a column called possibilty in which the vaule is 1 if pressure is lesser than 1000 hPa

count_no_sub = len(y[y==0])
count_sub = len(y[y==1])
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("Percentage of instances with no occurence of hurricane ", pct_of_no_sub*100)
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("Percentage of instances where hurricane actually occured", pct_of_sub*100)

Percentage of instances with no occurence of hurricane  97.19536364182332
Percentage of instances where hurricane actually occured 2.804636358176686


In [104]:
clf = KNN(contamination=pct_of_sub/100, n_neighbors=10)


In [105]:

clf.fit(X)

KNN(algorithm='auto', contamination=0.0002804636358176686, leaf_size=30,
  method='largest', metric='minkowski', metric_params=None, n_jobs=1,
  n_neighbors=10, p=2, radius=1.0)

In [106]:
# Get the prediction labels of the training data
y_train_pred = clf.labels_ 

# Outlier scores
y_train_scores = clf.decision_scores_
print(y_train_scores)

[0.2236068  0.36055513 0.28284271 ... 0.14142136 0.14142136 0.14142136]


In [107]:
# Import the utility function for model evaluation
from pyod.utils import evaluate_print
# Evaluate on the training data
evaluate_print('KNN', y_train_pred, y_train_scores)


KNN ROC:1.0, precision @ rank n:1.0


In [108]:
print(y_train_scores)

[0.2236068  0.36055513 0.28284271 ... 0.14142136 0.14142136 0.14142136]


In [109]:
for i in range(len(y_train_pred)):
    df['possibility'][i]=a[i]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [110]:
df.to_csv("combined.csv")